In [11]:
!pip install gdown
!gdown --id 1F3C7qwol6TnvdTP3dRys1s90BV_uMONk
!unzip lemondetect.zip
!mv /lemondetection /content/


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1F3C7qwol6TnvdTP3dRys1s90BV_uMONk
To: /content/lemondetect.zip
100% 358M/358M [00:07<00:00, 49.5MB/s]
Archive:  lemondetect.zip
   creating: lemondetect/
  inflating: __MACOSX/._lemondetect  
  inflating: lemondetect/.DS_Store   
  inflating: __MACOSX/lemondetect/._.DS_Store  
   creating: lemondetect/test/
  inflating: __MACOSX/lemondetect/._test  
   creating: lemondetect/train/
  inflating: __MACOSX/lemondetect/._train  
   creating: lemondetect/test/diseased/
  inflating: __MACOSX/lemondetect/test/._diseased  
  inflating: lemondetect/test/.DS_Store  
  inflating: __MACOSX/lemondetect/test/._.DS_Store  
   creating: lemon

In [12]:
import tensorflow as tf
import IPython.display as display
import PIL
import PIL.Image
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import pathlib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [16]:
data_dir = pathlib.Path('lemondetect/train')
test_dir = pathlib.Path('lemondetect/test')
train_count = len(list(data_dir.glob('*/*.JPG')))
test_count = len(list(test_dir.glob('*/*.JPG')))

batch_size = 32
img_height = 180
img_width = 180

#Generate More Data by Generator
image_gen_train = ImageDataGenerator(rescale = 1./255,validation_split=0.2,rotation_range = 45,width_shift_range=.15,height_shift_range =.15,horizontal_flip=True,zoom_range=0.5)

#Shuffle All Data 
train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'training',  directory= data_dir,shuffle=True,target_size=(img_height, img_width))  

#Divide Data for Validation
validation_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,subset = 'validation',  directory= data_dir,shuffle=True,target_size=(img_height, img_width)) 

CLASS_NAMES = np.array([item.name for item in data_dir.glob('*') if item.name != "LICENSE.txt"])

Found 164 images belonging to 2 classes.
Found 41 images belonging to 2 classes.


In [20]:
num_classes = 2
model = Sequential([
  Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width ,3)),
  MaxPooling2D(),
  Conv2D(32, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Conv2D(64, 3, padding='same', activation='relu'),
  MaxPooling2D(),
  Dropout(0.2),
  Flatten(),
  Dense(128, activation='relu'),
  Dense(num_classes)
])
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.summary()
model.save('lemondetect/model.h5')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 180, 180, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 90, 90, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 90, 90, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 45, 45, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 45, 45, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 22, 22, 64)      

In [22]:
model_mango = load_model('lemondetect/model.h5')
model_mango.
image_gen_test = ImageDataGenerator(rescale = 1./255)
test_data_gen = image_gen_test.flow_from_directory(batch_size=32,directory= test_dir,shuffle=True,target_size=(img_height, img_width))  

start_time = time.time()
predictions = model_mango.predict(test_data_gen)
for i in range(len(predictions)):
    if(predictions[i][0]<0):
      print("Healthy     "+str(abs(predictions[i][0]))+"%")
    else:  
      print("Diseased    "+str(100-abs(predictions[i][0]))+"%")
print("Total Time: ", time.time()-start_time)
print("Average Time Per Image: ", (time.time()-start_time)/len(predictions))

Found 31 images belonging to 2 classes.
1/1 [==============================] - 12s 12s/step
Healthy     0.06495371%
Healthy     0.041907202%
Healthy     0.04783242%
Healthy     0.034866836%
Healthy     0.052886132%
Healthy     0.051080484%
Healthy     0.053802356%
Healthy     0.04683337%
Healthy     0.044691645%
Healthy     0.026878817%
Healthy     0.025511896%
Healthy     0.049889978%
Healthy     0.041140046%
Healthy     0.063894525%
Healthy     0.059166763%
Healthy     0.044740617%
Healthy     0.04473241%
Healthy     0.03258872%
Healthy     0.056582812%
Healthy     0.03775095%
Healthy     0.0598173%
Healthy     0.052074857%
Healthy     0.041315034%
Healthy     0.060007542%
Healthy     0.06599606%
Healthy     0.043384414%
Healthy     0.039292675%
Healthy     0.050851285%
Healthy     0.05498308%
Healthy     0.068006665%
Healthy     0.07262882%
Total Time:  32.53866219520569
Average Time Per Image:  1.0496403863353114


In [24]:
!pip install anvil-uplink
import anvil.server
anvil.server.connect("SNO5N6UNOGXMICYSINU4DIEJ-AFBDS2UOAU7ZPMEU")
import anvil.media
import numpy as np

from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import save_img
from tensorflow.keras.utils import img_to_array

import cv2
%cd /content
!mkdir tst
%cd tst
!mkdir test
%cd /content
@anvil.server.callable
def imgload(file):
  model_mango = load_model('lemondetect/model.h5')
  with anvil.media.TempFile(file) as filename:
    img = load_img(filename)
    img_array = img_to_array(img)
    save_img('/content/tst/test/temp.png',img_array)
    print("Processed Incoming Image")
    
  image_gen_test = ImageDataGenerator(rescale = 1./255)
  test_data_gen = image_gen_test.flow_from_directory(batch_size=1,directory= "/content/tst",shuffle=True,target_size=(img_height, img_width)) 
  predictions = model_mango.predict(test_data_gen) 
  print(test_data_gen,predictions)
  li=''
  if(predictions[0][0]<0):
      li='Healthy'
  else:  
    li="Diseased"
  return(li)
anvil.server.wait_forever()  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached anvil_uplink-0.4.2-py2.py3-none-any.whl (90 kB)
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Using cached ws4py-0.5.1-py3-none-any.whl


Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER
/content
/content/tst
/content
Processed Incoming Image
Found 1 images belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
<keras.preprocessing.image.DirectoryIterator object at 0x7f75171cadd0> [[-0.05608292 -0.03926367]]
Processed Incoming Image
Found 1 images belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
<keras.preprocessing.image.DirectoryIterator object at 0x7f75171a1240> [[-0.09109155 -0.02989144]]


KeyboardInterrupt: ignored